In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
#     ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds    
    ('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    ('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    ('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
#     ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
#     ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    #('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4446


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(10, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0039
Function value obtained: 0.4731
Current minimum: 0.4731
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0022
Function value obtained: 0.4736
Current minimum: 0.4731
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0027
Function value obtained: 0.4724
Current minimum: 0.4724
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0020
Function value obtained: 0.4736
Current minimum: 0.4724
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.7121
Function value obtained: 0.4738
Current minimum: 0.4721
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.6832
Function value obtained: 0.4742
Current minimum: 0.4721
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.8173
Function value obtained: 0.4722
Current minimum: 0.4721
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.7462
Function value obtained: 0.4735
Current minimum: 0.4721
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.7891
Function value obtained: 0.4742
Current minimum: 0.4721
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 1.0273
Function value obtained: 0.4729
Current minimum: 0.4719
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 1.0785
Function value obtained: 0.4739
Current minimum: 0.4719
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 1.2139
Function value obtained: 0.4742
Current minimum: 0.4719
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 1.1754
Function value obtained: 0.4736
Current minimum: 0.4719
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 1.1146
Function value obtained: 0.4720
Current minimum: 0.4719
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.5501
Function value obtained: 0.4371
Current minimum: 0.4371
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.5800
Function value obtained: 0.4379
Current minimum: 0.4371
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.5084
Function value obtained: 0.4373
Current minimum: 0.4371
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.4921
Function value obtained: 0.4380
Current minimum: 0.4371
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.4886
Function value obtained: 0.4386
Current minimum: 0.4371
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 1.0221
Function value obtained: 0.4378
Current minimum: 0.4370
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 1.1958
Function value obtained: 0.4378
Current minimum: 0.4370
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 1.1407
Function value obtained: 0.4370
Current minimum: 0.4370
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 1.0763
Function value obtained: 0.4370
Current minimum: 0.4370
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 1.2884
Function value obtained: 0.4371
Current minimum: 0.4370
Iteration No: 66 started. Sea

...

Iteration No: 37 ended. Search finished for the next optimal point.
Time taken: 0.6487
Function value obtained: 0.4628
Current minimum: 0.4628
Iteration No: 38 started. Searching for the next optimal point.
Iteration No: 38 ended. Search finished for the next optimal point.
Time taken: 1.0398
Function value obtained: 0.4627
Current minimum: 0.4627
Iteration No: 39 started. Searching for the next optimal point.
Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 0.8614
Function value obtained: 0.4646
Current minimum: 0.4627
Iteration No: 40 started. Searching for the next optimal point.
Iteration No: 40 ended. Search finished for the next optimal point.
Time taken: 0.8215
Function value obtained: 0.4655
Current minimum: 0.4627
Iteration No: 41 started. Searching for the next optimal point.
Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 1.1027
Function value obtained: 0.4666
Current minimum: 0.4627
Iteration No: 42 started. Sea

Iteration No: 77 ended. Search finished for the next optimal point.
Time taken: 1.1091
Function value obtained: 0.4626
Current minimum: 0.4626
Iteration No: 78 started. Searching for the next optimal point.
Iteration No: 78 ended. Search finished for the next optimal point.
Time taken: 1.1154
Function value obtained: 0.4652
Current minimum: 0.4626
Iteration No: 79 started. Searching for the next optimal point.
Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 1.0378
Function value obtained: 0.4649
Current minimum: 0.4626
Iteration No: 80 started. Searching for the next optimal point.
Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 1.0365
Function value obtained: 0.4640
Current minimum: 0.4626
Iteration No: 81 started. Searching for the next optimal point.
Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 1.1261
Function value obtained: 0.4646
Current minimum: 0.4626
Iteration No: 82 started. Sea

Iteration No: 16 ended. Search finished for the next optimal point.
Time taken: 0.3999
Function value obtained: 0.4384
Current minimum: 0.4376
Iteration No: 17 started. Searching for the next optimal point.
Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 0.3008
Function value obtained: 0.4384
Current minimum: 0.4376
Iteration No: 18 started. Searching for the next optimal point.
Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 0.3386
Function value obtained: 0.4381
Current minimum: 0.4376
Iteration No: 19 started. Searching for the next optimal point.
Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 0.3441
Function value obtained: 0.4386
Current minimum: 0.4376
Iteration No: 20 started. Searching for the next optimal point.
Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 0.3767
Function value obtained: 0.4375
Current minimum: 0.4375
Iteration No: 21 started. Sea

Iteration No: 56 ended. Search finished for the next optimal point.
Time taken: 1.0963
Function value obtained: 0.4384
Current minimum: 0.4372
Iteration No: 57 started. Searching for the next optimal point.
Iteration No: 57 ended. Search finished for the next optimal point.
Time taken: 1.1243
Function value obtained: 0.4378
Current minimum: 0.4372
Iteration No: 58 started. Searching for the next optimal point.
Iteration No: 58 ended. Search finished for the next optimal point.
Time taken: 1.1976
Function value obtained: 0.4373
Current minimum: 0.4372
Iteration No: 59 started. Searching for the next optimal point.
Iteration No: 59 ended. Search finished for the next optimal point.
Time taken: 0.8549
Function value obtained: 0.4372
Current minimum: 0.4372
Iteration No: 60 started. Searching for the next optimal point.
Iteration No: 60 ended. Search finished for the next optimal point.
Time taken: 0.9577
Function value obtained: 0.4381
Current minimum: 0.4372
Iteration No: 61 started. Sea

Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 1.4488
Function value obtained: 0.4380
Current minimum: 0.4369
Iteration No: 97 started. Searching for the next optimal point.
Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 1.3620
Function value obtained: 0.4369
Current minimum: 0.4369
Iteration No: 98 started. Searching for the next optimal point.
Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 1.4044
Function value obtained: 0.4385
Current minimum: 0.4369
Iteration No: 99 started. Searching for the next optimal point.
Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 1.4247
Function value obtained: 0.4382
Current minimum: 0.4369
Iteration No: 100 started. Searching for the next optimal point.
Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 1.5497
Function value obtained: 0.4379
Current minimum: 0.4369
[0.07226848 0.0268383  0.02

In [10]:
best_weights

{'cohesion': array([0.10842239, 0.07928643, 0.02148524, 0.11688647, 0.02148524,
        0.21485236, 0.0605729 , 0.07822416, 0.08393246, 0.21485236]),
 'syntax': array([0.14045722, 0.02135712, 0.04047927, 0.16731126, 0.02135712,
        0.21357121, 0.0576102 , 0.1316197 , 0.11977459, 0.08646231]),
 'vocabulary': array([0.22195005, 0.02389647, 0.02389647, 0.23896474, 0.09161283,
        0.10257481, 0.04782636, 0.0656167 , 0.04476057, 0.13890099]),
 'phraseology': array([0.19068878, 0.01906888, 0.01906888, 0.19068878, 0.01906888,
        0.19068878, 0.01906888, 0.11248734, 0.1095501 , 0.1296207 ]),
 'grammar': array([0.09174326, 0.02394496, 0.02394496, 0.22708532, 0.02394496,
        0.02394496, 0.08254733, 0.02394496, 0.23944964, 0.23944964]),
 'conventions': array([0.07226848, 0.0268383 , 0.0268383 , 0.06679992, 0.03205081,
        0.13453239, 0.0268383 , 0.0770674 , 0.26838305, 0.26838305])}

In [11]:
scores

{'cohesion': 0.4719195003287524,
 'syntax': 0.4369010693500467,
 'vocabulary': 0.40483798841040586,
 'phraseology': 0.44675742842951044,
 'grammar': 0.4626051297936887,
 'conventions': 0.43687048676968465}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4433


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.945595,0.914074,0.910519,0.825765,0.888777
syntax,0.945595,1.000000,0.938799,0.960701,0.917670,0.929281
vocabulary,0.914074,0.938799,1.000000,0.959776,0.876470,0.892207
phraseology,0.910519,0.960701,0.959776,1.000000,0.943540,0.882183
grammar,0.825765,0.917670,0.876470,0.943540,1.000000,0.855318
conventions,0.888777,0.929281,0.892207,0.882183,0.855318,1.000000
